### Here i will implement messages, chatLLms, Tools

In [7]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [8]:

from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint ### for printing in good way

messages=[AIMessage(content="hey hi how are you doing, how can i help you",name="LLM")]
messages.append(HumanMessage(content="what is the date and time now",name="user"))
messages.append(AIMessage(content="what is your location",name="LLM"))
messages.append(HumanMessage(content="iam in missouri, saint louis",name="user"))

messages

[AIMessage(content='hey hi how are you doing, how can i help you', additional_kwargs={}, response_metadata={}, name='LLM'),
 HumanMessage(content='what is the date and time now', additional_kwargs={}, response_metadata={}, name='user'),
 AIMessage(content='what is your location', additional_kwargs={}, response_metadata={}, name='LLM'),
 HumanMessage(content='iam in missouri, saint louis', additional_kwargs={}, response_metadata={}, name='user')]

In [16]:
for message in messages:
    message.pretty_print()

================================== Ai Message ==================================
Name: LLM

hey hi how are you doing, how can i help you
================================ Human Message =================================
Name: user

what is the date and time now
================================== Ai Message ==================================
Name: LLM

what is your location
================================ Human Message =================================
Name: user

iam in missouri, saint louis


###now with llm models

In [23]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4o")
llm.invoke(messages)

AIMessage(content="I can't provide real-time information, including the current date and time. However, you can easily find out the current date and time on your device or through an online search. Since Missouri is in the Central Time Zone (CST/CDT), you can use that as a reference.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 57, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_37d212baff', 'id': 'chatcmpl-Cl2WUVzjWNprIfLBy8pGFU5Y684Bv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--a1861934-0640-453b-af2e-128c4ad3bf8c-0', usage_metadata={'input_tokens': 57, 'output_tokens': 57, 'total_tokens': 114, 'input_token_det

### Tool Calling

In [24]:
llm

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x0000026868A53C80>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000026868A52E70>, root_client=<openai.OpenAI object at 0x0000026868A51160>, root_async_client=<openai.AsyncOpenAI object at 0x0000026868A53440>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [28]:
def divide(a:int, b: int)->int:
    return a/b


In [31]:
llm_tools=llm.bind_tools([divide])
tool_call=llm_tools.invoke("what is 10 divided by 5")
tool_call.tool_calls

[{'name': 'divide',
  'args': {'a': 10, 'b': 5},
  'id': 'call_Uen393qQ8J4ETWpvCBvi9yu8',
  'type': 'tool_call'}]

In [4]:
### using messages as states

from langchain_core.messages import AnyMessage
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from typing import Annotated

In [5]:
def State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]


In [12]:
prev_messages=[AIMessage(content="hey hi how are you doing, how can i help you",name="LLM")]
prev_messages.append(HumanMessage(content="what is the date and time now",name="user"))

ai_msg=AIMessage(content="what is your location",name="LLM")

In [13]:
add_messages(prev_messages,ai_msg)

[AIMessage(content='hey hi how are you doing, how can i help you', additional_kwargs={}, response_metadata={}, name='LLM', id='fdd73e21-c6dd-403f-9933-266e3f5d3a53'),
 HumanMessage(content='what is the date and time now', additional_kwargs={}, response_metadata={}, name='user', id='35317c97-56bf-4c9b-93db-4689ad096aff'),
 AIMessage(content='what is your location', additional_kwargs={}, response_metadata={}, name='LLM', id='e02f1db0-42c3-46a1-8cd1-0a66e43a0f0d')]